In [1]:
%uv pip install datasets transformers peft trl

Using Python 3.10.19 environment at: /usr/local
Resolved 68 packages in 39ms
Uninstalled 1 package in 1ms
Installed 1 package in 59ms
 - fsspec==2025.12.0
 + fsspec==2025.10.0
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Data

In [3]:
from datasets import load_dataset

dataset = load_dataset(
    "json",
    data_files={
        "train": "new_data1.json",
    }
)


Generating train split: 10000 examples [00:00, 16182.55 examples/s]


In [4]:
data = dataset["train"]
data

Dataset({
    features: ['Document', 'Summary'],
    num_rows: 10000
})

In [5]:
#train test split
data = data.train_test_split(test_size=0.1)
train_data = data['train']
test_data = data['test']


In [6]:
print(len(train_data))
print(len(test_data))

9000
1000


In [7]:
train_data[0]

{'Document': 'Đặt một bàn tay lên gốc cây lan để che miệng chậu, tay kia cầm chậu cây và nhẹ nhàng lật cây lan lại vào bàn tay đỡ cây lan.  Nếu cây lan bám vào chậu, bạn hãy lay nhẹ tới lui để cây long ra. Chỉ cắt rễ hoặc cành cây nếu bạn không thể lắc cho cây long ra khỏi chậu. Nếu phải cắt cây, bạn nên cố gắng bảo tồn rễ và cành càng nhiều càng tốt. Vẫn cẩn thận cầm cây lan bằng một tay, tay kia bạn bóc giá thể cũ được càng nhiều càng tốt. Khi đã loại bỏ được các mảng giá thể lớn, bạn hãy rửa rễ cây bằng nước ấm để loại bỏ phần còn lại. Việc loại bỏ hoàn toàn giá thể cũ sẽ giúp cây lan có thể hút tối đa dưỡng chất khi được trồng lại vào chậu mới, đồng thời đảm bảo tiêu diệt hết sâu bọ. Khi cây lan đã được rửa sạch, bạn hãy kiểm tra kỹ lưỡng để tìm các lá, cành, rễ và củ bẹ chết. Dùng dụng cụ đã khử trùng cắt bỏ những phần rễ mềm và nâu, lá cây úa vàng và các củ bẹ đen quắt.  Củ bẹ là một đặc điểm ở một số loài lan. Củ bẹ là bộ phận có hình củ hành mọc gần gốc cây mà từ đó lá cây sẽ m

In [20]:
%uv pip install -U bitsandbytes accelerate transformers

Using Python 3.10.19 environment at: /usr/local
Resolved 43 packages in 69ms
Prepared 1 package in 0.64ms
Uninstalled 1 package in 1ms
Installed 1 package in 56ms
 - fsspec==2025.10.0
 + fsspec==2025.12.0
Note: you may need to restart the kernel to use updated packages.


In [8]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    # bnb_4bit_use_double_quant=False,
)

# 2. Load model và tokenizer
model_name = "Qwen/Qwen3-4B-Instruct-2507"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    cache_dir="./base_model_cache"
)

Loading checkpoint shards: 100%|█████████████████████████████████████████████████| 3/3 [00:05<00:00,  1.74s/it]


In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, use_fast = False)

In [10]:
tokenizer


Qwen2Tokenizer(name_or_path='Qwen/Qwen3-4B-Instruct-2507', vocab_size=151643, model_max_length=1010000, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '<|im_end|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|im_start|>', '<|im_end|>', '<|object_ref_start|>', '<|object_ref_end|>', '<|box_start|>', '<|box_end|>', '<|quad_start|>', '<|quad_end|>', '<|vision_start|>', '<|vision_end|>', '<|vision_pad|>', '<|image_pad|>', '<|video_pad|>']}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	151643: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151644: AddedToken("<|im_start|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151645: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151646: AddedToken("<|object_ref_start|>", rstrip=False, lstrip=False, single_word=False, 

In [11]:
def format_prompt(sample):
    article = sample['Document'].strip()
    summary = sample['Summary'].strip()
    messages = [
        {"role": "system", "content": "Bạn là một biên tập viên báo chí chuyên nghiệp. Nhiệm vụ của bạn là tóm tắt bài báo được cung cấp. "},
        {"role": "user", "content": article},
        {"role": "assistant", "content": summary}
    ]

    formatted_text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
    return formatted_text
        

In [12]:

print(format_prompt(train_data[0]))

<|im_start|>system
Bạn là một biên tập viên báo chí chuyên nghiệp. Nhiệm vụ của bạn là tóm tắt bài báo được cung cấp. <|im_end|>
<|im_start|>user
Đặt một bàn tay lên gốc cây lan để che miệng chậu, tay kia cầm chậu cây và nhẹ nhàng lật cây lan lại vào bàn tay đỡ cây lan.  Nếu cây lan bám vào chậu, bạn hãy lay nhẹ tới lui để cây long ra. Chỉ cắt rễ hoặc cành cây nếu bạn không thể lắc cho cây long ra khỏi chậu. Nếu phải cắt cây, bạn nên cố gắng bảo tồn rễ và cành càng nhiều càng tốt. Vẫn cẩn thận cầm cây lan bằng một tay, tay kia bạn bóc giá thể cũ được càng nhiều càng tốt. Khi đã loại bỏ được các mảng giá thể lớn, bạn hãy rửa rễ cây bằng nước ấm để loại bỏ phần còn lại. Việc loại bỏ hoàn toàn giá thể cũ sẽ giúp cây lan có thể hút tối đa dưỡng chất khi được trồng lại vào chậu mới, đồng thời đảm bảo tiêu diệt hết sâu bọ. Khi cây lan đã được rửa sạch, bạn hãy kiểm tra kỹ lưỡng để tìm các lá, cành, rễ và củ bẹ chết. Dùng dụng cụ đã khử trùng cắt bỏ những phần rễ mềm và nâu, lá cây úa vàng và

## Lora config

In [13]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                   "gate_proj", "up_proj", "down_proj",],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

In [14]:
training_args = TrainingArguments(
    output_dir="./results/qwen-lora-summarization",
    num_train_epochs=4,
    per_device_train_batch_size=4,  # Điều chỉnh theo VRAM của bạn
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    warmup_steps=100,
    weight_decay=0.01,
    learning_rate=5e-5,
    bf16=True,  # Thay bằng fp16=True nếu GPU không hỗ trợ bf16
    fp16=False,
    logging_steps=20,
    eval_strategy="steps",
    eval_steps=200,
    save_steps=200,
    save_total_limit=3,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    report_to = 'none',
    optim = 'paged_adamw_32bit'
)

In [15]:
trainer = SFTTrainer(
    model=model,
    processing_class=tokenizer,
    train_dataset=train_data,
    eval_dataset=test_data,
    args=training_args,
    peft_config=lora_config,
    formatting_func=format_prompt,
    # max_seq_length=512,
    # packing=True,  # Tăng hiệu suất training
)

Truncating eval dataset: 100%|██████████████████████████████████| 1000/1000 [00:00<00:00, 115085.86 examples/s]
Detected kernel version 4.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [16]:
trainer.train()

# 9. Lưu model
output_dir = "./final-lora-model"
trainer.model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"\nModel đã lưu tại: {output_dir}")

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


Step,Training Loss,Validation Loss,Entropy,Num Tokens,Mean Token Accuracy
200,1.661100,1.624548,1.628125,1254935.000000,0.621200
400,1.620500,1.590141,1.604408,2504420.000000,0.626947
600,1.600600,1.572178,1.592088,3769621.000000,0.629989
800,1.586500,1.560602,1.572962,5023543.000000,0.631879
1000,1.587000,1.552068,1.573938,6278472.000000,0.633564
1200,1.565700,1.545771,1.536791,7543878.000000,0.634596
1400,1.524500,1.541566,1.538361,8799515.000000,0.635473
1600,1.524500,1.537109,1.552449,10064850.000000,0.636163
1800,1.557200,1.532644,1.543285,11310255.000000,0.637150
2000,1.494400,1.530351,1.526271,12575140.000000,0.637897



Model đã lưu tại: ./final-lora-model


In [25]:
# ...existing code...
# Chọn một mẫu bất kỳ từ tập test (ví dụ mẫu đầu tiên)
sample = test_data[400]
document = sample['Document']
ground_truth = sample['Summary']

# Tạo messages cho inference (Lưu ý: Không bao gồm phần 'assistant' chứa tóm tắt mẫu)
messages = [
    {"role": "system", "content": "Bạn là một biên tập viên báo chí chuyên nghiệp. Nhiệm vụ của bạn là tóm tắt bài báo được cung cấp. "},
    {"role": "user", "content": document}
]

# Tokenize input và đưa lên thiết bị (GPU)
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

# Sinh văn bản (Generate)
outputs = model.generate(
    input_ids,
    max_new_tokens = 666,
    temperature=0.1,         # Độ sáng tạo
    top_p=0.9,
    do_sample=True,
    pad_token_id=tokenizer.eos_token_id
)

# Decode kết quả (cắt bỏ phần prompt input ban đầu để chỉ lấy phần model sinh ra)
response = tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True)

print("-" * 50)
print(f"Văn bản gốc:\n{document}")
print("-" * 50)
print(f"Tóm tắt gốc (Ground Truth):\n{ground_truth}")
print("-" * 50)
print(f"Mô hình dự đoán:\n{response}")
print("-" * 50)

--------------------------------------------------
Văn bản gốc (500 ký tự đầu):
Trong năm 2016 , ước tính 600.000 trẻ em đã tử vong do nhiễm trùng đường hô hấp dưới cấp tính gây ra bởi không khí ô nhiễm .Ô nhiễm không khí là một trong những mối đe doạ hàng đầu đối với sức khoẻ ở trẻ em dưới 5 tuổi , chiếm gần 1 trong 10 ca tử vong ở nhóm tuổi này , báo cáo cho biết .Theo WHO , trẻ em dễ bị ô nhiễm hơn vì chiều cao thấp hơn người trưởng thành nên sẽ hít vào cơ thể các chất độc hại gần mặt đất nhiều hơn người trưởng thành .Tại các nước thu nhập thấp và trung bình , 98 % trẻ em dưới 5 tuổi thường xuyên tiếp xúc với bụi siêu vi PM 2.5 .Còn ở các nước có thu nhập cao , 52 % trẻ em dưới 5 tuổi tiếp xúc với PM 2.5 .Tiến sĩ Maria Neira , giám đốc Sở Y tế công cộng , môi trường và xã hội ( WHO ) , cho biết : " Ô nhiễm không khí làm não trẻ em chậm phát triển , ảnh hưởng đến sức khoẻ của chúng theo nhiều cách hơn chúng ta biết " .Tổ chức WHO đưa ra một loạt hành động cần thiết để hạn chế vấn đề 